<a href="https://colab.research.google.com/github/drushti-surkar/Hydro-Map.ai/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy geopandas shapely scikit-learn flask gradio
!apt-get update
!apt-get install -y sqlite3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.8 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit

In [ ]:

import sqlite3


conn = sqlite3.connect('NagrikNiti.db')

conn.execute("DROP TABLE IF EXISTS flow_measurements")
conn.execute("DROP TABLE IF EXISTS predictions")

conn.execute("""
    CREATE TABLE flow_measurements (
        measurement_id TEXT PRIMARY KEY,
        user_id INTEGER,
        flow_rate FLOAT,
        lat FLOAT,
        lon FLOAT,
        timestamp TEXT
    )
""")

conn.execute("""
    CREATE TABLE predictions (
        prediction_id TEXT PRIMARY KEY,
        lat FLOAT,
        lon FLOAT,
        probability FLOAT,
        timestamp TEXT
    )
""")
conn.commit()
conn.close()

print("NagrikNiti database initialized!")

NagrikNiti database initialized!


In [ ]:
import pandas as pd
import numpy as np
import sqlite3


np.random.seed(42)
n_points = 100
lat = np.random.uniform(12.9, 13.1, n_points)  # Chennai coordinates
lon = np.random.uniform(80.1, 80.3, n_points)
flow_rate = np.random.normal(0.5, 0.1, n_points)  # Normal flow ~0.5 L/s
anomalies = np.random.choice(n_points, 10)  # 10% anomalies
flow_rate[anomalies] = np.random.uniform(0.01, 0.15, 10)  # Very low flow for leaks


data = pd.DataFrame({
    'user_id': np.arange(n_points),
    'flow_rate': flow_rate,
    'lat': lat,
    'lon': lon,
    'timestamp': pd.Timestamp.now()
})


conn = sqlite3.connect('NagrikNiti.db')

conn.execute("DROP TABLE IF EXISTS flow_measurements")

conn.execute("""
    CREATE TABLE flow_measurements (
        measurement_id TEXT PRIMARY KEY,
        user_id INTEGER,
        flow_rate FLOAT,
        lat FLOAT,
        lon FLOAT,
        timestamp TEXT
    )
""")
for i, row in data.iterrows():
    conn.execute(
        "INSERT INTO flow_measurements (measurement_id, user_id, flow_rate, lat, lon, timestamp) VALUES (?, ?, ?, ?, ?, ?)",
        (str(i), row['user_id'], row['flow_rate'], row['lat'], row['lon'], str(row['timestamp']))
    )
conn.commit()
conn.close()

print("Synthetic data loaded successfully for NagrikNiti!")

Synthetic data loaded successfully for NagrikNiti!


In [ ]:
import sqlite3
import pandas as pd
from sklearn.ensemble import IsolationForest
from datetime import datetime
import uuid

def submit_flow(user_id, flow_rate, lat, lon):
    try:
        timestamp = datetime.now()
        measurement_id = str(uuid.uuid4())

        conn = sqlite3.connect('NagrikNiti.db')
        conn.execute(
            "INSERT INTO flow_measurements (measurement_id, user_id, flow_rate, lat, lon, timestamp) VALUES (?, ?, ?, ?, ?, ?)",
            (measurement_id, user_id, flow_rate, lat, lon, str(timestamp))
        )
        conn.commit()
        conn.close()

        health_score = "Normal" if flow_rate >= 0.3 else "Low"
        return {
            "status": "success",
            "health_score": health_score,
            "message": "Flow rate submitted successfully for NagrikNiti!"
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

def predict():
    try:
        conn = sqlite3.connect('NagrikNiti.db')
        query = "SELECT user_id, flow_rate, lat, lon, timestamp FROM flow_measurements"
        data = pd.read_sql(query, conn)
        conn.close()

        print("NagrikNiti: Number of rows fetched:", len(data))
        print("NagrikNiti: Flow rates:", data['flow_rate'].describe())


        if len(data) < 10:
            return {"status": "error", "message": f"Insufficient data: Only {len(data)} rows available. Need at least 10 rows for anomaly detection."}

        # Anomaly detection
        iso_forest = IsolationForest(contamination=0.15, random_state=42)  # Adjusted contamination
        data['anomaly'] = iso_forest.fit_predict(data[['flow_rate']])
        anomalies = data[data['anomaly'] == -1]

        print("NagrikNiti: Number of anomalies detected:", len(anomalies))
        print("NagrikNiti: Anomalies:", anomalies[['flow_rate', 'lat', 'lon']].head().to_dict())

        # Store predictions
        conn = sqlite3.connect('NagrikNiti.db')
        for _, row in anomalies.iterrows():
            prediction_id = str(uuid.uuid4())
            conn.execute(
                "INSERT INTO predictions (prediction_id, lat, lon, probability, timestamp) VALUES (?, ?, ?, ?, ?)",
                (prediction_id, row['lat'], row['lon'], 0.9, str(datetime.now()))
            )
        conn.commit()
        conn.close()

        # Return predictions
        predictions = [
            {"lon": row['lon'], "lat": row['lat'], "probability": 0.9}
            for _, row in anomalies.iterrows()
        ]
        return predictions
    except Exception as e:
        return {"status": "error", "message": str(e)}

# Test the functions
print("Testing NagrikNiti submit_flow:")
print(submit_flow(user_id=101, flow_rate=0.4, lat=13.0, lon=80.2))
print("\nTesting NagrikNiti predict:")
print(predict())

Testing NagrikNiti submit_flow:
{'status': 'success', 'health_score': 'Normal', 'message': 'Flow rate submitted successfully for NagrikNiti!'}

Testing NagrikNiti predict:
NagrikNiti: Number of rows fetched: 101
NagrikNiti: Flow rates: count    101.000000
mean       0.461396
std        0.163864
min        0.012015
25%        0.414284
50%        0.501300
75%        0.557089
max        0.885273
Name: flow_rate, dtype: float64
NagrikNiti: Number of anomalies detected: 15
NagrikNiti: Anomalies: {'flow_rate': {15: 0.05983617501117662, 21: 0.01570203232465582, 25: 0.023696782491140204, 27: 0.7720169166589619, 57: 0.8852731490654722}, 'lat': {15: 12.936680901970687, 21: 12.927898772130408, 25: 13.057035192278603, 27: 13.002846887682722, 57: 12.93919657248383}, 'lon': {15: 80.27429211803754, 21: 80.2792182599847, 25: 80.18542155772525, 27: 80.27214611665127, 57: 80.25232392306575}}
[{'lon': 80.27429211803754, 'lat': 12.936680901970687, 'probability': 0.9}, {'lon': 80.2792182599847, 'lat': 12.9

In [ ]:
!pip install gradio

import gradio as gr
import sqlite3
import pandas as pd
import uuid
from datetime import datetime

def gradio_submit_flow(user_id, flow_rate, lat, lon):
    result = submit_flow(int(user_id), float(flow_rate), float(lat), float(lon))
    return f"Health Score: {result['health_score']}\nMessage: {result['message']}"

def gradio_predict():
    predictions = predict()
    if isinstance(predictions, dict) and predictions.get("status") == "error":
        return predictions["message"]
    if not predictions:
        return "No leaks detected in NagrikNiti."

    html = """
    <div id="map" style="width: 100%; height: 400px;"></div>
    <script src="https://unpkg.com/leaflet@1.9.3/dist/leaflet.js"></script>
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.3/dist/leaflet.css" />
    <script>
        var map = L.map('map').setView([13.0, 80.2], 12);
        L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
            attribution: '© OpenStreetMap'
        }).addTo(map);
    """
    for pred in predictions:
        html += f"L.circle([{pred['lat']}, {pred['lon']}], {{color: 'red', radius: 100}}).addTo(map);"
    html += "</script>"
    return html


with gr.Blocks() as demo:
    gr.Markdown("# NagrikNiti: Water Leakage Detection System")
    with gr.Tab("Citizen Form"):
        gr.Markdown("Submit your water flow rate to help detect leaks with NagrikNiti.")
        user_id = gr.Textbox(label="User ID", placeholder="Enter your ID (e.g., 101)")
        flow_rate = gr.Textbox(label="Flow Rate (L/s)", placeholder="Enter flow rate (e.g., 0.4)")
        lat = gr.Textbox(label="Latitude", placeholder="Enter latitude (e.g., 13.0)")
        lon = gr.Textbox(label="Longitude", placeholder="Enter longitude (e.g., 80.2)")
        submit_btn = gr.Button("Submit")
        output = gr.Textbox(label="Result")
        submit_btn.click(gradio_submit_flow, inputs=[user_id, flow_rate, lat, lon], outputs=output)
    with gr.Tab("Municipal Dashboard"):
        gr.Markdown("View predicted leak zones on a map with NagrikNiti.")
        predict_btn = gr.Button("Show Predictions")
        map_output = gr.HTML(label="Leak Zones Map")
        predict_btn.click(gradio_predict, outputs=map_output)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c8774116e4045e263b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
